In [1]:
import cv2
import numpy as np
import mediapipe as mp
import os

class VideoToNpyConverter:
    def __init__(self, seq_len=37, max_hands=2):
        self.seq_len = seq_len
        self.max_hands = max_hands
        self.landmark_dim = 21 * 3 * max_hands  # 126 for 2 hands

        mp_hands = mp.solutions.hands
        self.hands = mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=max_hands,
            min_detection_confidence=0.6,
            min_tracking_confidence=0.6
        )

    def extract_landmarks(self, frame):
        """Extracts up to 2-hand landmarks (126 dims)."""
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(frame_rgb)

        if not results.multi_hand_landmarks:  # no hand detected
            return np.zeros(self.landmark_dim)

        coords = []

        # process up to 2 hands
        for hand in results.multi_hand_landmarks[:self.max_hands]:
            for lm in hand.landmark:
                coords.extend([lm.x, lm.y, lm.z])

        # pad if only 1 hand found
        if len(results.multi_hand_landmarks) < self.max_hands:
            coords.extend([0.0] * (21 * 3))

        return np.array(coords)

    def normalize_sequence(self, seq):
        """Normalize to fixed seq_len using interpolation."""
        if len(seq) == 0:
            return np.zeros((self.seq_len, self.landmark_dim))

        indices = np.linspace(0, len(seq) - 1, self.seq_len).astype(int)
        return np.array([seq[i] for i in indices])

    def video_to_npy(self, video_path):
        """Convert video file → numpy array (seq_len × 126)."""
        cap = cv2.VideoCapture(video_path)

        if not cap.isOpened():
            raise ValueError(f"Cannot open video: {video_path}")

        sequence = []

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            lm_vec = self.extract_landmarks(frame)
            sequence.append(lm_vec)

        cap.release()

        # normalize to fixed length
        sequence = self.normalize_sequence(sequence)

        return sequence

    def save(self, video_path, output_dir):
        """Convert and save as .npy."""
        os.makedirs(output_dir, exist_ok=True)

        seq = self.video_to_npy(video_path)

        base = os.path.splitext(os.path.basename(video_path))[0]
        out_path = os.path.join(output_dir, base + ".npy")

        np.save(out_path, seq)
        print(f"Saved: {out_path}")

        return out_path


In [ ]:
from tqdm import tqdm

converter = VideoToNpyConverter(seq_len=37)

root = "data\\SL_retained_classes"
out_root = "data\\SL_retained_classes_npy"

os.makedirs(out_root, exist_ok=True)

all_videos = []
for cls in os.listdir(root):
    cls_path = os.path.join(root, cls)
    if not os.path.isdir(cls_path):
        continue
    
    out_dir = os.path.join(out_root, cls)
    os.makedirs(out_dir, exist_ok=True)
    
    for vid in os.listdir(cls_path):
        if vid.lower().endswith((".mp4", ".avi")):
            full_path = os.path.join(cls_path, vid)
            all_videos.append((full_path, out_dir))

print(f"Processing {len(all_videos)} videos...")

for video_path, out_dir in tqdm(all_videos, desc="Converting videos", ncols=100):
    try:
        converter.save(video_path, out_dir)
    except Exception as e:
        print(f"\nERROR: {video_path} - {e}")

<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
<>:3: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\S'
C:\Users\ASUS\AppData\Local\Temp\ipykernel_45320\3289078510.py:3: SyntaxWarning: invalid escape sequence '\S'
  root = "data\SL_retained_classes"
C:\Users\ASUS\AppData\Local\Temp\ipykernel_45320\3289078510.py:4: SyntaxWarning: invalid escape sequence '\S'
  out_root = "data\SL_retained_classes_npy"


Saved: data\SL_retained_classes_npy\a\01610.npy
Saved: data\SL_retained_classes_npy\a\01611.npy
Saved: data\SL_retained_classes_npy\a\01612.npy
Saved: data\SL_retained_classes_npy\a\01615.npy
Saved: data\SL_retained_classes_npy\a\66039.npy
Saved: data\SL_retained_classes_npy\abdomen\00335.npy
Saved: data\SL_retained_classes_npy\abdomen\00336.npy
Saved: data\SL_retained_classes_npy\abdomen\00338.npy
Saved: data\SL_retained_classes_npy\abdomen\00339.npy
Saved: data\SL_retained_classes_npy\abdomen\00341.npy
Saved: data\SL_retained_classes_npy\adapt\00943.npy
Saved: data\SL_retained_classes_npy\adapt\00944.npy
Saved: data\SL_retained_classes_npy\adapt\00946.npy
Saved: data\SL_retained_classes_npy\adapt\00947.npy
Saved: data\SL_retained_classes_npy\adapt\00949.npy
Saved: data\SL_retained_classes_npy\adapt\00951.npy
Saved: data\SL_retained_classes_npy\adjective\01064.npy
Saved: data\SL_retained_classes_npy\adjective\01065.npy
Saved: data\SL_retained_classes_npy\adjective\01066.npy
Saved: dat